# Merge All Job Data Sources

This notebook merges job data from multiple scrapers (XpressJobs, LinkedIn, TopJobs, Ikman) into a single master dataset: `data/processed/all_jobs_master.csv`.


In [6]:
import pandas as pd
from pathlib import Path
import glob
import os


PROJECT_ROOT = Path("..").resolve()
RAW_DIR = PROJECT_ROOT / "data" / "raw" / "jobs"
PROCESSED_DIR = PROJECT_ROOT / "data" / "processed"
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

print(f"Reading from: {RAW_DIR}")
print(f"Saving to: {PROCESSED_DIR}")

Reading from: C:\Users\User\Desktop\Second Year Stuff\sdgp\PathFinder+\Adjusted_Scraper\project_root\data\raw\jobs
Saving to: C:\Users\User\Desktop\Second Year Stuff\sdgp\PathFinder+\Adjusted_Scraper\project_root\data\processed


In [7]:


def standardize_columns(df, source_name):
    df = df.copy()
    
    # Define mappings based on source

    if source_name == 'XpressJobs':
        # Expected: title, company, location, description, job_url, scraped_date
        mapping = {
            'job_url': 'url',
            'scraped_date': 'date'
        }
        data_type = 'Live'
    elif source_name == 'LinkedIn':
        # Expected: title, company_name, location, description, job_url, posted_date
        mapping = {
            'title': 'title',
            'company_name': 'company',
            'job_url': 'url',
            'posted_date': 'date'
        }
        data_type = 'Training' # Per user request: LinkedIn data is for training models, not live application
    elif source_name == 'TopJobs':
        # Expected: title, company, description, date? job_url?
        # TopJobs might vary, check your specific file
        mapping = {
            'opening_date': 'date',
            'url': 'url'
        }
        data_type = 'Live'
    elif source_name == 'Ikman':
        # Expected: job_title, company, description, etc.
        mapping = {
            'job_title': 'title',
            'company_name': 'company',
            'job_url': 'url',
            'posted_date': 'date'
        }
        data_type = 'Live'
    else:
        mapping = {}
        data_type = 'Unknown'

    # Renaissance logic: Rename available columns
    df.rename(columns=mapping, inplace=True)
    
    # Ensure required columns exist
    required = ['title', 'company', 'location', 'description', 'date', 'url']
    for col in required:
        if col not in df.columns:
            df[col] = None  # Fill missing with None
            
    # Add source and type columns
    df['source'] = source_name
    df['data_type'] = data_type
    
    # Return only the standard columns
    return df[required + ['source', 'data_type']]

In [8]:

all_dataframes = []

# -- XpressJobs --
xpress_files = glob.glob(str(RAW_DIR / "xpressjobs_ALL_CATEGORIES_CLEAN_*.csv"))
for f in xpress_files:
    try:
        df = pd.read_csv(f)
        df = standardize_columns(df, 'XpressJobs')
        all_dataframes.append(df)
        print(f"Loaded XpressJobs: {len(df)} rows")
    except Exception as e:
        print(f"Failed XpressJobs {f}: {e}")

# -- LinkedIn --
linkedin_files = glob.glob(str(RAW_DIR / "linkedin_sri_lanka_IT_jobs.csv"))
for f in linkedin_files:
    try:
        df = pd.read_csv(f)
        df = standardize_columns(df, 'LinkedIn')
        all_dataframes.append(df)
        print(f"Loaded LinkedIn: {len(df)} rows")
    except Exception as e:
        print(f"Failed LinkedIn {f}: {e}")

# -- TopJobs --
topjobs_files = glob.glob(str(RAW_DIR / "topjobs_FINAL_*.csv"))
for f in topjobs_files:
    try:
        df = pd.read_csv(f)
        df = standardize_columns(df, 'TopJobs')
        all_dataframes.append(df)
        print(f"Loaded TopJobs: {len(df)} rows")
    except Exception as e:
        print(f"Failed TopJobs {f}: {e}")
        
# -- Ikman --
ikman_files = glob.glob(str(RAW_DIR / "ikman_*.csv"))
for f in ikman_files:
    try:
        df = pd.read_csv(f)
        df = standardize_columns(df, 'Ikman')
        all_dataframes.append(df)
        print(f"Loaded Ikman: {len(df)} rows")
    except Exception as e:
        print(f"Failed Ikman {f}: {e}")

In [9]:

if all_dataframes:
    master_df = pd.concat(all_dataframes, ignore_index=True)
    print(f"\nTotal rows loaded: {len(master_df)}")
    
    # Deduplicate by URL (if exists) or Title+Company
    # Some legacy scraped/manual data might not have URLs
    
    # Strategy: 
    # 1. Drop exact duplicates
    master_df.drop_duplicates(inplace=True)
    
    # 2. Drop rows with NO title
    master_df = master_df.dropna(subset=['title'])
    
    # 3. Drop duplicates on URL where URL is not null
    # We separate rows with URL and rows without
    with_url = master_df[master_df['url'].notna()]
    no_url = master_df[master_df['url'].isna()]
    
    with_url = with_url.drop_duplicates(subset=['url'], keep='last')
    
    # Re-combine
    final_df = pd.concat([with_url, no_url], ignore_index=True)
    
    print(f"Final unique count: {len(final_df)}")
    
    # Save
    output_path = PROCESSED_DIR / "all_jobs_master.csv"
    final_df.to_csv(output_path, index=False)
    print(f"Saved to {output_path}")
    
    # Show Distribution
    print("\n--- Combined Data Distribution ---")
    print(final_df['data_type'].value_counts())
    print("\n")
    print(final_df['source'].value_counts())
else:
    print("No data loaded!")

No data loaded!


In [10]:

if 'final_df' in locals():
    display(final_df.sample(5))